# Installation

Clone the repo
```
git clone https://github.com/aubreymoore/CRB-Damage-Dataset-Improvement
```

Move to the new folder
```
cd CRB-Damage-Dataset-Improvement
```

Create a virtual environment
```
python3 -m venv .venv
```

Activate the new virtual environment
```
source venv/bin/activate
```

Install required python modules
```
pip install -r code/requirements.txt
```

Create a .gitignore file and add .venv to the list of files and folders to be ignored.
Adding a virtual environmant to a repository is bad practice.
```
echo ".venv" >> .gitignore
```

# References

https://pybit.es/articles/a-better-place-to-put-your-python-virtual-environments/

[Image Deduplication](https://github.com/voxel51/fiftyone-examples/blob/master/examples/image_deduplication.ipynb)

[CVAT <> FiftyOne: Data-Centric Machine Learning with Two Open Source Tools](https://www.cvat.ai/post/data-centric)

[FiftyOne - Ultralytics Integration](https://docs.voxel51.com/integrations/ultralytics.html)

[Finding Detection Mistakes with FiftyOne](https://docs.voxel51.com/tutorials/detection_mistakes.html)

[Fine-tune YOLOv8 models for custom use cases with the help of FiftyOne](https://docs.voxel51.com/tutorials/yolov8.html)

[FiftyOne Brain](https://docs.voxel51.com/brain.html)

In [1]:
import os
import shutil
import glob
import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import logging
import sys
from icecream import ic
from datetime import datetime
import numpy as np
from numpy.linalg import norm
from ultralytics import YOLO


In [2]:
def add_timestamp_field():
    dataset.add_sample_field("timestamp", fo.DateTimeField)

    for sample in dataset:
        timestamp_str = os.path.basename(sample.filepath)[4:-4]
        dt = datetime.strptime(timestamp_str, '%Y%m%d_%H%M%S')
        # ic(timestamp_str, dt)
        sample['timestamp'] = dt
        sample.save()
    
    # Create view  
    view = dataset.sort_by(F'timestamp')
    dataset.save_view('sorted_by_timestamp', view, overwrite=True)

In [3]:
def update_requirements_file():
    os.system('pip list --format=freeze > requirements.txt')

# update_requirements_file()

In [4]:
def create_new_dataset(original_ds_path, new_ds_path):
    """ 
    """
    os.mkdir(new_ds_path)
    os.mkdir(f'{new_ds_path}/images')
    os.mkdir(f'{new_ds_path}/images/train')
    os.mkdir(f'{new_ds_path}/images/val')
    os.mkdir(f'{new_ds_path}/labels')
    os.mkdir(f'{new_ds_path}/labels/train')
    os.mkdir(f'{new_ds_path}/labels/val')
    
    for filepath in glob.glob(f'{original_ds_path}/train/*.jpg'):
        shutil.copy2(filepath, f'{new_ds_path}/images/train')
    for filepath in glob.glob(f'{original_ds_path}/train/*.txt'):
        shutil.copy2(filepath, f'{new_ds_path}/labels/train')
    for filepath in glob.glob(f'{original_ds_path}/val/*.jpg'):
        shutil.copy2(filepath, f'{new_ds_path}/images/val')
    for filepath in glob.glob(f'{original_ds_path}/val/*.txt'):
        shutil.copy2(filepath, f'{new_ds_path}/labels/val')
        
    s = f'path: {new_ds_path} \n'
    s += 'train: ./images/train/ \n'
    s += 'val: ./images/val/ \n'
    s += 'names: \n'
    s += '  0: live \n'
    s += '  1: dead \n'
    s += '  2: vcut \n'
    with open(f'{new_ds_path}/dataset.yaml', 'w') as f:
        f.write(s)

In [5]:
def yolo2fiftyone(fo_dataset_name, dataset_dir, splits=["train", "val"]):
    """ 
    Imports a dataset in YOLO5 format into FiftyOne, using tags to mark the samples in each split 
    """ 
    dataset = fo.Dataset(name, persistent=True)
    for split in splits:
        dataset.add_dir(
            dataset_dir=dataset_dir,
            dataset_type=fo.types.YOLOv5Dataset,
            split=split,
            tags=split,
    )
    return dataset

In [6]:
def add_embeddings_field():
    """ 
    """ 
    model = foz.load_zoo_model("mobilenet-v2-imagenet-torch")
    dataset.compute_embeddings(model=model, embeddings_field='embeddings')

In [7]:
def cosine_similarity(a, b):
    return np.dot(a,b)/(norm(a)*norm(b))
 
# a = np.array([2,1,2,3,2,9])
# b = np.array([3,4,2,4,5,5])
# cosine_similarity(a, b)

In [8]:
def add_similarity_with_prev_img_field():
    """ 
    """
    view = dataset.load_saved_view('sorted_by_timestamp')
    # thresh = 0.92
    first_sample = True
    for sample in view:
        if first_sample:
            current_embeddings = sample.embeddings
            similarity = 0.0
            first_sample = False
        else:
            previous_embeddings = current_embeddings
            current_embeddings = sample.embeddings
            similarity = cosine_similarity(previous_embeddings, current_embeddings)
        sample['similarity_with_prev_img'] = similarity
        # if similarity > thresh:
        #     sample.tags.append(f'similarity>{thresh}')
        # else:
        #     sample.tags.append('similarity OK') 
        sample.save()

In [9]:
def add_predictions_field():
    """ 
    """
    # Load YOLOv8 model
    # from ultralytics import YOLO
    model = YOLO(ORIGINAL_MODEL_PATH)
    dataset.apply_model(model, label_field="yolov8")
    
# add_predictions_field()

In [10]:
def add_mistakenness_field():
    """ 
    Adds mistakenness, possible_missing and possible_spurious fields.
    See docs at https://docs.voxel51.com/brain.html#label-mistakes for details.
    """
    fob.compute_mistakenness(dataset, "yolov8", label_field="ground_truth")  
    
# add_mistakenness_field() 

In [11]:
def add_field(fieldname, func):
    """ 
    This utility function checks for existence of a field in a dataset.
    If the field does not exist it is added by running func.
    """
    if dataset.get_field(fieldname):
        logger.info(f'"{fieldname}" field already exists')
    else:
        logger.info(f'Adding "{fieldname}" field')
        func()

# def add_new_field():
#     """ 
#     Code for adding a field named 'new' should be inserted in this function.
#     """
#     pass
    
# add_field('new', add_new_field)

In [12]:
def create_autocorrelated_images_view(threshold, delete=False):
    """ 
    """
    dataset = fo.load_dataset(fo_dataset_name)
    sorted_by_timestamp_view = dataset.load_saved_view('sorted_by_timestamp')
    view = sorted_by_timestamp_view.match(
        F('similarity_with_prev_img') > threshold)
    dataset.save_view("autocorrelated_images_view", view, overwrite=True)
    count = view.count()
    
    if delete:
        dataset.delete_samples(view) 
        dataset.save()
     
    return count
  
# create_autocorrelated_images_view(0.98, True)

In [13]:
def count_ground_truth_bbs(dataset):
    total_detections = 0
    for sample in dataset:
        total_detections += len(sample.ground_truth.detections)
    return total_detections

# count_ground_truth_bbs()

In [14]:
def create_bb_touching_edge_view(delete=False):
    """ 
    https://docs.voxel51.com/recipes/remove_duplicate_annos.html
    """
    dataset = fo.load_dataset(fo_dataset_name)
    view = dataset.filter_labels('ground_truth', 
        (F('bounding_box')[0] <= 0) | # left
        (F('bounding_box')[1] <= 0) | # top
        ((F('bounding_box')[0] + F('bounding_box')[3]) >= 1) # right
    )
    dataset.save_view('bb_touching_edge', view, overwrite=True) 
    count = view.count()
           
    if delete:
        dataset.delete_labels(view)
    dataset.save()
            
    return  count

# create_bb_touching_edge_view()

In [15]:
def configure_logger(logfile):
    """
    Configure logger to send messages to notebook and logfile
    """
    logging.root.handlers = []
    logging.basicConfig(
        level=logging.INFO, 
        format='%(asctime)s %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
            logging.FileHandler(filename=logfile),
            logging.StreamHandler(sys.stdout)
        ]
    )
    logger = logging.getLogger()
    return logger

In [16]:
# MAIN

# Constants

original_ds_path = '/home/aubrey/Desktop/Guam07-training-set/datasets/3class-no-symlinks'
ORIGINAL_MODEL_PATH = f'{original_ds_path}/runs/detect/train5/weights/best.pt'
new_ds_path = '/home/aubrey/datasets/Guam07v3'
fo_dataset_name = 'Guam07v3'
logfile = 'create_new_dataset.log'
AUTOCORRELATED_IMAGES_THRESHOLD = 0.98
DELETE_AUTOCORRELATED_IMAGES = True
DELETE_BBS_TOUCHING_EDGES = True

#configure logger
logger = configure_logger(logfile)

# update requirements.txt
logger.info('Updating "requirements.txt"')
update_requirements_file()

# wrangle dataset into YOLOv5 format
if os.path.exists(new_ds_path):
    logger.info(f'"{new_ds_path}" already exists in YOLOv5 format')
else:
    logger.info(f'creating dataset "{new_ds_path}" in YOLOv5 format')
    create_new_dataset(original_ds_path, new_ds_path)

# Create new FiftyOne dataset
if fo_dataset_name in fo.list_datasets():
    logger.info(f'FiftyOne dataset "{fo_dataset_name}" already exists') 
else:
    logger.info(f'Creating FiftyOne dataset "{fo_dataset_name}"')
    dataset = yolo2fiftyone(name=fo_dataset_name, dataset_dir=new_ds_path)
    
# Load dataset
logger.info(f'Loading FiftyOne dataset "{fo_dataset_name}"')
dataset = fo.load_dataset(fo_dataset_name)
logger.info(f'Ground truth bounding boxes: {count_ground_truth_bbs(dataset)}')

# Add fields if they don't already exist
add_field('timestamp', add_timestamp_field)
add_field('embeddings', add_embeddings_field)
add_field('similarity_with_prev_img', add_similarity_with_prev_img_field)
add_field('yolov8', add_predictions_field)
add_field('mistakenness', add_mistakenness_field)

# Find bounding boxes touching left, top or right edges of images
logger.info('Creating "bb_touching_edge_view"')
if DELETE_BBS_TOUCHING_EDGES:
    logger.info('    "DELETE_BBS_TOUCHING_EDGES" is True; bbs will be deleted')
else:
    logger.info('    "DELETE_BBS_TOUCHING_EDGES" is False; bbs will not be deleted')
bb_touching_edge_count = create_bb_touching_edge_view(DELETE_BBS_TOUCHING_EDGES)
logger.info(f'    {bb_touching_edge_count} ground truth bounding boxes touching image edges were found')

# Find autocorrelated images
logger.info('Creating "autocorrelated_images_view"')
if DELETE_BBS_TOUCHING_EDGES:
    logger.info('    "DELETE_AUTOCORRELATED_IMAGES" is True; samples will be deleted')
else:
    logger.info('    "DELETE_AUTOCORRELATED_IMAGES" is False; bbs will not be deleted')
autocorrelated_image_count = create_autocorrelated_images_view(
    threshold=AUTOCORRELATED_IMAGES_THRESHOLD, delete=DELETE_AUTOCORRELATED_IMAGES)
logger.info(f'With a threshold of {AUTOCORRELATED_IMAGES_THRESHOLD}, {autocorrelated_image_count} autocorrelated images were found')

# Reoad dataset
logger.info(f'Loading FiftyOne dataset "{fo_dataset_name}"')
dataset = fo.load_dataset(fo_dataset_name)
logger.info(f'Ground truth bounding boxes: {count_ground_truth_bbs(dataset)}')

# Launch FiftyOne app in browser
logger.info(f'Launching FifyOne app in browser')
session = fo.launch_app(dataset, auto=False)
logger.info(session)

logger.info('FINISHED')

2024-11-24 16:24:42 Updating "requirements.txt"
2024-11-24 16:24:42 "/home/aubrey/datasets/Guam07v3" already exists in YOLOv5 format
2024-11-24 16:24:44 FiftyOne dataset "Guam07v3" already exists
2024-11-24 16:24:44 Loading FiftyOne dataset "Guam07v3"
2024-11-24 16:24:51 Ground truth bounding boxes: 14807
2024-11-24 16:24:51 "timestamp" field already exists
2024-11-24 16:24:51 "embeddings" field already exists
2024-11-24 16:24:51 "similarity_with_prev_img" field already exists
2024-11-24 16:24:51 "yolov8" field already exists
2024-11-24 16:24:51 "mistakenness" field already exists
2024-11-24 16:24:51 Creating "bb_touching_edge_view"
2024-11-24 16:24:51     "DELETE_BBS_TOUCHING_EDGES" is True; bbs will be deleted
2024-11-24 16:24:51     0 ground truth bounding boxes touching image edges were found
2024-11-24 16:24:51 Creating "autocorrelated_images_view"
2024-11-24 16:24:51     "DELETE_AUTOCORRELATED_IMAGES" is True; samples will be deleted
2024-11-24 16:24:51 With a threshold of 0.98, 

In [17]:
# dataset.info['notes'] = 'Here is another note about this dataset.' 
# dataset.save()
# dataset.info